In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tripod_files = glob.glob('./npy/Tripod*.npy')
tripod_files

In [ ]:
data1 = np.load(tripod_files[0])
data2 = np.load(tripod_files[1])
data3 = np.load(tripod_files[2])

In [ ]:
plt.plot(data1)
plt.plot(data2)
plt.plot(data3)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import datetime

# Fixing random state for reproducibility
np.random.seed(19680801)

# tick every 5th easter
#rule = rrulewrapper(YEARLY, byeaster=1, interval=1)

#loc = RRuleLocator(rule)

formatter = DateFormatter('%m-%d')
date1 = datetime.date(2019, 5, 22)
date2 = datetime.date(2019, 7, 5)
delta = datetime.timedelta(days=1)

dates = drange(date1, date2, delta)

#s = np.random.rand(len(dates))  # make up some random y values

fig, ax = plt.subplots()

plt.plot_date(dates, data1, '.')
#ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_tick_params(rotation=30, labelsize=10)
ax.set_ylabel('IoU')

plt.show()

In [ ]:
len(dates)

In [ ]:
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *

import pprint
import sys
sys.displayhook = pprint.pprint

In [ ]:
x_axis = list(rrule(DAILY, interval=1, dtstart=parse("20190522T090000"), until=parse("20190705T000000")))

In [ ]:
len(x_axis)

In [ ]:
dates